# First Attempt towards sentiment analysis
#### I've used the same model that I tried for MNIST dataset. I got 60% accuracy, proves that modification is to be done in order to observe improvisation. 
#### Inference: Need on work on bigger training data, maybe 1.6 million sentences. Or maybe somewhat different algorithm, tweak with hidden layers, etc.

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
import numpy as np
import pickle

Let's open the file which we pickled back in the day.

In [3]:
with open("/Users/tejasreddy9/Documents/PyNotebooks/sentiment_set.pickle","rb") as file:
    train_x, train_y, test_x, test_y = pickle.load(file)

As we loaded in the training and validation sets, let's do the same process as we have done to MNIST dataset.

Now dimension will be `train_x[i]` initially. In the case of MNIST dataset, it was `784`(28*28).

In [4]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 2
batch_size = 100

initial_size = len(train_x[0])
x = tf.placeholder('float', [None, initial_size])
y = tf.placeholder('float')

Network model is the same with the core concept of feedforward-backpropagation (epoch) model.

Let's use the core concept of deep learning, computations are done by setting weights for each connection of one in a layer to the other in the next layer. A bias term is added to ensure that the result is not always zero whenever data or weights is zero. If the result from a layer is all zero for certain entries, then it would be biased for that layer as the contribution of the following layers is very insignificant.

This is the computation to be done for each layer. (weights * data) + biases

Let's use random_normal function for generating random values. We use ReLU activation function on each layer.

Feedforward model as the output of one layer is given as input to the next layer.

In [6]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([initial_size, n_nodes_hl1])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])), 
                        'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)  # activation function 	
    
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output_layer['weights']), output_layer['biases'])
    return output

Now, given that our model is implemented, let's train over training data.

The cost function we consider here is reduced mean of the softmax cross entropy results over the labels.

So, now that done, we go into running our cycles. Each cycle (epoch) consists of a feedforward stride and a backpropagtion result used to improve the weights internally. Also for each epoch, we calculate corresponding loss after using AdamOptimizer to reduce minimize the cost function. We'll display the progress.

By comparing which predictions actual match, accuracy is calculated by reduced mean. Now, let's evaluate over the completely unknown test data.

In [7]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(0,len(train_x),batch_size):
                batch_x = np.array(train_x[i:i+batch_size])
                batch_y = np.array(train_y[i:i+batch_size])
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
                epoch_loss += c
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))


Let's run that. It goes through each epoch and we can observe the model building by reducing the cost function, self adjusting the weights.

In [8]:
train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch 0 completed out of 10 loss: 228661.16357421875
Epoch 1 completed out of 10 loss: 116298.01574707031
Epoch 2 completed out of 10 loss: 71950.77996826172
Epoch 3 completed out of 10 loss: 48113.845764160156
Epoch 4 completed out of 10 loss: 33231.18165588379
Epoch 5 completed out of 10 loss: 31761.54453277588
Epoch 6 completed out of 10 loss: 30378.60150909424
Epoch 7 completed out of 10 loss: 15416.030141830444
Epoch 8 completed out of 10 loss: 10187.167241096497
Epoch 9 completed out of 10 loss: 8538.833990573883
Accuracy: 0.57973737


Not much impressive, I must try another approach or better dataset.